# ---------------------------------------------------------
# CREATE SUMMARY DATAFRAME WITH DIFFERENCE
# ---------------------------------------------------------

# Select the two countries
main_country = "Philippines"
compare_country = "Japan"

# Extract rows for these countries
main_row = df[df["Country"] == main_country]
compare_row = df[df["Country"] == compare_country]

# Initialize summary dict
summary_data = {
    "Metric": ["National Debt (USD)", "Debt % GDP", "Debt Per Capita"],
    main_country: [
        int(main_row["Debt USD"]) if not main_row.empty else None,
        float(main_row["Debt % GDP"]) if not main_row.empty else None,
        int(main_row["Debt Per Capita"]) if not main_row.empty else None
    ],
    compare_country: [
        int(compare_row["Debt USD"]) if not compare_row.empty else None,
        float(compare_row["Debt % GDP"]) if not compare_row.empty else None,
        int(compare_row["Debt Per Capita"]) if not compare_row.empty else None
    ],
    "Difference": [
        (int(main_row["Debt USD"]) - int(compare_row["Debt USD"])) if not main_row.empty and not compare_row.empty else None,
        (float(main_row["Debt % GDP"]) - float(compare_row["Debt % GDP"])) if not main_row.empty and not compare_row.empty else None,
        (int(main_row["Debt Per Capita"]) - int(compare_row["Debt Per Capita"])) if not main_row.empty and not compare_row.empty else None
    ]
}

# Create the DataFrame
summary_df = pd.DataFrame(summary_data)

# Print the summary DataFrame
print("\n--- Country Debt Comparison ---")
print(summary_df)
